In [2]:
%pip install pandas
%pip install numpy
%pip install spacy
%pip install -U scikit-learn
%pip install seaborn
%pip install transformers
%pip install fasttext
%pip install lazypredict
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393294 sha256=08e87ad1a2c5fefbbba06fc37234aa4d4303a4f80b8e3021c427e7766cb56495
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold
import fasttext
import fasttext.util
from transformers import BertTokenizer
import gensim
import gensim.downloader

In [6]:
davidson = pd.read_csv("davidson_sentiment_score.csv")
davidson

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,care.virtue,care.vice,fairness.virtue,...,loyalty.virtue,loyalty.vice,authority.virtue,authority.vice,sanctity.virtue,sanctity.vice,cleaned_tweets,positive,neutral,negative
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,0,0,0,...,0,0,0,0,1,1,rt mayasolovely woman complain clean house am...,0.0679,0.4113,0.5208
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,0,0,0,...,0,0,0,0,0,0,rt mleew boy dat cold tyga dwn bad cuffin d...,0.0326,0.4309,0.5365
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,0,0,0,...,0,0,0,0,0,2,rt urkindofbrand dawg rt sbaby life fuck b...,0.0031,0.0268,0.9701
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,0,0,0,...,0,0,0,0,0,0,rt c g anderson viva based look like tranny,0.0609,0.7310,0.2081
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,0,0,0,...,0,0,0,0,0,1,rt shenikaroberts shit hear true faker bitch ...,0.0119,0.1853,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,0,0,0,...,0,0,0,0,1,1,muthaf in lie lifeasking pearls c...,0.0087,0.1202,0.8711
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an...",0,0,0,...,0,0,0,0,0,0,go break wrong heart baby drive redneck crazy,0.0087,0.1022,0.8891
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...,0,0,0,...,0,0,0,0,0,0,young buck wanna eat dat nigguh like be nt fuc...,0.0389,0.3873,0.5738
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies,0,0,0,...,0,0,0,0,0,0,youu got wild bitch tellin lie,0.0096,0.1096,0.8808


In [23]:
from pathlib import Path

def getTrainSetFastText():
    ft_model = fasttext.load_model('fasttext_model.bin')
    traindata = []
    for tweet in davidson['cleaned_tweets']:
        traindata.append(ft_model.get_sentence_vector(tweet.replace("\n", "")))
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetTFIDF():
    countvec = CountVectorizer(max_features=100)
    bow = countvec.fit_transform(davidson['tweet']).toarray()
    tfidfconverter = TfidfTransformer()
    X = tfidfconverter.fit_transform(bow).toarray()
    training_data = pd.DataFrame(X)
    training_data.columns = training_data.columns.astype(str)
    training_data["positive"] = davidson["positive"].values.tolist()
    training_data["neutral"] = davidson["neutral"].values.tolist()
    training_data["negative"] = davidson["negative"].values.tolist()
    return training_data

def getTrainSetBERT():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenized_data = tokenizer(davidson['cleaned_tweets'].tolist(), padding='max_length', max_length=100)
    traindata = []
    for msg in tokenized_data['input_ids']:
        traindata.append(msg)
    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetWord2Vec():
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec-google-news-300.bin', binary=True)
    traindata = []
    for msg in davidson['tweet']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in w2v_model:
                vecs.append(w2v_model[word][:100])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)
    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

def getTrainSetGlove():
    glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
    traindata = []
    for msg in davidson['cleaned_tweets']:
        words = msg.split()
        vecs = []
        for word in words:
            if word in glove_vectors:
                vecs.append(glove_vectors[word])
        if vecs:
            vec_avg = sum(vecs) / len(vecs)
        else:
            vec_avg = [0] * 100
        traindata.append(vec_avg)

    traindata = pd.DataFrame(traindata)
    traindata.columns = traindata.columns.astype(str)

    traindata["positive"] = davidson["positive"].values.tolist()
    traindata["neutral"] = davidson["neutral"].values.tolist()
    traindata["negative"] = davidson["negative"].values.tolist()
    return traindata

In [24]:
X = getTrainSetFastText()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.049839,-0.012389,-0.023938,0.020247,0.039021,0.000894,-0.035659,0.014219,-0.007426,-0.001875,...,0.037399,-0.046555,0.007986,0.050923,0.043329,0.003237,0.004343,0.0679,0.4113,0.5208
1,-0.036568,0.003346,0.033397,0.013426,0.030098,0.017432,-0.029294,-0.018229,0.038597,0.026475,...,-0.016421,0.034347,-0.006825,0.049985,0.011514,0.028711,0.029309,0.0326,0.4309,0.5365
2,-0.033726,-0.026513,0.051228,0.023314,-0.008450,-0.007304,-0.002611,-0.044960,-0.021767,0.029252,...,-0.018170,0.007356,0.025731,-0.022734,-0.031255,0.010993,0.006138,0.0031,0.0268,0.9701
3,0.016069,-0.023210,0.017003,0.020582,0.020618,-0.082636,0.031038,-0.041252,-0.050100,-0.016178,...,0.071336,-0.090715,0.037946,-0.038263,0.062424,0.014616,-0.024076,0.0609,0.7310,0.2081
4,-0.039872,0.003142,-0.052659,-0.010374,0.019240,-0.054540,0.042485,0.009805,-0.032044,0.017162,...,0.019592,0.058020,0.027119,-0.024716,-0.033323,-0.031128,0.028149,0.0119,0.1853,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.027526,-0.008089,-0.041829,0.011872,-0.017959,-0.062028,0.004057,-0.010666,-0.031026,0.003157,...,-0.014292,-0.018228,0.019039,0.017356,-0.014026,0.050776,-0.024824,0.0087,0.1202,0.8711
24779,0.074124,0.055645,0.003045,-0.062518,0.018502,0.023498,0.028821,-0.003021,-0.028707,0.017454,...,-0.002361,0.017819,-0.057399,0.027355,0.015336,-0.029446,-0.016158,0.0087,0.1022,0.8891
24780,-0.045470,-0.020662,0.007163,0.030336,0.013641,0.004268,0.012252,0.017398,0.011659,-0.046131,...,-0.059007,0.008888,-0.020042,0.011536,0.092346,-0.033128,-0.025090,0.0389,0.3873,0.5738
24781,0.045859,0.034949,0.032117,-0.026320,-0.077364,-0.005260,0.020944,-0.048831,0.045265,0.021779,...,0.028218,-0.018713,0.087524,-0.011091,-0.016091,0.010496,0.096376,0.0096,0.1096,0.8808


In [26]:
X.to_csv("fasttext_embedded_davidson.csv")

In [28]:
X = getTrainSetBERT()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,101,19387,9815,19454,21818,2135,2450,17612,4550,2160,...,0,0,0,0,0,0,0,0.0679,0.4113,0.5208
1,101,19387,19875,4402,2860,2879,23755,3147,5939,3654,...,0,0,0,0,0,0,0,0.0326,0.4309,0.5365
2,101,19387,24471,18824,11253,23544,4830,27767,19387,24829,...,0,0,0,0,0,0,0,0.0031,0.0268,0.9701
3,101,19387,1039,1043,5143,20022,2241,2298,2066,25283,...,0,0,0,0,0,0,0,0.0609,0.7310,0.2081
4,101,19387,21882,7556,3217,8296,2015,4485,2963,2995,...,0,0,0,0,0,0,0,0.0119,0.1853,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,101,14163,8322,2546,1999,4682,2166,19895,2075,21944,...,0,0,0,0,0,0,0,0.0087,0.1202,0.8711
24779,101,2175,3338,3308,2540,3336,3298,2417,18278,4689,...,0,0,0,0,0,0,0,0.0087,0.1022,0.8891
24780,101,2402,10131,10587,4521,23755,9152,13871,27225,2066,...,0,0,0,0,0,0,0,0.0389,0.3873,0.5738
24781,101,2017,2226,2288,3748,7743,2425,2378,4682,102,...,0,0,0,0,0,0,0,0.0096,0.1096,0.8808


In [30]:
X.to_csv("BERT_embedded_davidson.csv")

In [31]:
X = getTrainSetGlove()
X

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,-0.111302,0.306922,0.348273,-0.485753,-0.065798,0.548288,-0.280351,0.407172,-0.007857,0.261233,...,-0.041109,-0.282496,0.096682,-0.167010,-0.125292,0.493742,-0.170647,0.0679,0.4113,0.5208
1,0.092538,-0.015601,0.218761,-0.405492,-0.358004,0.087583,-0.057481,0.099665,0.062261,-0.075047,...,0.249358,0.121851,-0.222610,-0.185078,-0.051823,0.189029,-0.332290,0.0326,0.4309,0.5365
2,0.035538,0.078532,0.334371,-0.469615,-0.284483,0.481790,-0.203210,-0.014088,0.034459,-0.193352,...,0.190936,0.128705,-0.270672,-0.105754,0.013771,0.128548,-0.029961,0.0031,0.0268,0.9701
3,-0.079496,0.316734,0.111112,-0.393971,-0.004359,0.052196,0.066891,0.114490,-0.702883,-0.003566,...,0.028228,0.057921,0.033683,-0.102843,-0.279136,0.165071,-0.082354,0.0609,0.7310,0.2081
4,0.004079,0.060200,0.466768,-0.528709,-0.341708,0.244955,-0.059650,-0.066839,0.256315,-0.056666,...,-0.085211,0.033253,-0.306345,-0.374493,-0.239821,0.085747,-0.020030,0.0119,0.1853,0.8027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24778,0.004741,0.397857,0.198663,-0.381434,0.024961,0.430087,0.080138,0.042919,-0.092584,0.056110,...,0.049797,-0.209973,-0.085747,-0.309594,-0.251637,0.197287,0.383541,0.0087,0.1202,0.8711
24779,0.084244,0.123301,0.503534,-0.439359,-0.307696,0.406169,-0.059857,0.266795,0.137413,0.016222,...,-0.037261,-0.454166,-0.267817,-0.012690,-0.032479,0.217305,0.082315,0.0087,0.1022,0.8891
24780,-0.082361,0.277683,0.222919,-0.409922,-0.412597,0.169749,-0.022676,0.016909,0.324785,-0.200622,...,-0.174379,-0.096924,-0.390437,-0.123616,-0.223217,0.339955,0.099451,0.0389,0.3873,0.5738
24781,0.035826,0.128624,0.802240,-0.276008,-0.258737,0.264376,0.083066,0.059068,0.097960,-0.457222,...,-0.120048,-0.008804,-0.196296,-0.412728,-0.018859,0.049473,0.118611,0.0096,0.1096,0.8808


In [32]:
X.to_csv("glove_embedded_davidson.csv")